# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [1]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'
IN_MEMORY_API = True #Quando falso utiliza a API listada

## Datasets

Cria um dataset, emula a lib request para substituir o acesso ao endpoint, por dados em memoria e deriva os Ids para escrever o arquivo CSV

In [2]:
in_memory_users = [
    {
      "id": 1,
      "name": "Pyterlei",
      "account": {
      "id": 7,
      "number": "00001-1",
      "agency": "0001",
      "balance": 50.0,
      "limit": 500.0
      },
      "card": {
        "id": 1,
        "number": "**** **** **** 1111",
        "limit": 1000.0
      },
      "features": [],
      "news": []
    },
    {
      "id": 2,
      "name": "Pytano",
      "account": {
      "id": 7,
      "number": "00002-1",
      "agency": "0001",
      "balance": 500.0,
      "limit": 500.0
      },
      "card": {
        "id": 2,
        "number": "**** **** **** 1112",
        "limit": 1000.0
      },
      "features": [],
      "news": []
    },
    {
      "id": 3,
      "name": "Pytone",
      "account": {
      "id": 7,
      "number": "00003-1",
      "agency": "0001",
      "balance": 5000.0,
      "limit": 1000.0
      },
      "card": {
        "id": 3,
        "number": "**** **** **** 1113",
        "limit": 1000.0
      },
      "features": [],
      "news": []
    },
]

In [3]:
class Response:
  status_code = None
  _json = None

  def __init__(self, status, content):
    self.status_code = status
    self._json = content

  def json(self):
    return self._json

class InMemoryApi:
  _data = None

  def __init__(self, in_memory_data):
    self._data = in_memory_data

  def get(self, url):
    try:
      id = int(url.split('/')[-1])
      user = [x for x in self._data if x["id"] == id]
      if len(user) == 0:
        return Response(404,None)
      return Response(200,user[0])
    except:
      return Response(400,None)

  def put(self, url,*,json = None):
    try:
      id = int(url.split('/')[-1])
      user = [(idx, x) for idx, x in enumerate(self._data) if x["id"] == id]
      if len(user) == 0:
        self._data.append(json)
        return Response(200,self._data[-1])

      self._data[user[0][0]] = json
      return Response(200,self._data[user[0][0]])
    except:
      return Response(400,None)

In [4]:
import pandas as pd

user_list = [user["id"] for user in in_memory_users]
user_df = pd.DataFrame({"UserID": user_list})

display(user_df)

,UserID
0,1
1,2
2,3


In [5]:
user_df.to_csv("SDW2023.csv")

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [6]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3]


In [7]:
import json

if (IN_MEMORY_API):
  requests = InMemoryApi(in_memory_users)
else:
  import requests

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id).copy()) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1,
    "name": "Pyterlei",
    "account": {
      "id": 7,
      "number": "00001-1",
      "agency": "0001",
      "balance": 50.0,
      "limit": 500.0
    },
    "card": {
      "id": 1,
      "number": "**** **** **** 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 2,
    "name": "Pytano",
    "account": {
      "id": 7,
      "number": "00002-1",
      "agency": "0001",
      "balance": 500.0,
      "limit": 500.0
    },
    "card": {
      "id": 2,
      "number": "**** **** **** 1112",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3,
    "name": "Pytone",
    "account": {
      "id": 7,
      "number": "00003-1",
      "agency": "0001",
      "balance": 5000.0,
      "limit": 1000.0
    },
    "card": {
      "id": 3,
      "number": "**** **** **** 1113",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Utilize a API do OpenAI GPT-3.5 para gerar uma mensagem de marketing personalizada para cada usuário, com base no seu patrimônio atual.

In [8]:
!pip install openai

In [9]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'TODO'

In [10]:
def resource_analysis(acount_balance):
  if acount_balance < 100:
    return "pequeno"
  elif acount_balance < 1000:
    return "medio"
  else:
    return "grande"

In [11]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  resources = resource_analysis(user["account"]["balance"])
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para o cliente {user['name']} que tem um {resources} patrimonio sobre a importância dos investimentos (máximo de 150 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Pyterlei, invista no seu futuro! Os investimentos são essenciais para o crescimento do seu patrimônio e realização de seus sonhos. Não deixe para depois!
Pytano, invista agora para proteger e fazer crescer seu patrimônio. Garanta seu futuro financeiro com as melhores opções de investimento!
Proteja seu patrimônio e faça seu dinheiro trabalhar para você. Invista de forma inteligente com a Pytone, sua aliada financeira.


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [12]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Pyterlei updated? True!
User Pytano updated? True!
User Pytone updated? True!


##Verificar os dado em memória

In [13]:
in_memory_users

[{'id': 1,
  'name': 'Pyterlei',
  'account': {'id': 7,
   'number': '00001-1',
   'agency': '0001',
   'balance': 50.0,
   'limit': 500.0},
  'card': {'id': 1, 'number': '**** **** **** 1111', 'limit': 1000.0},
  'features': [],
  'news': [{'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg',
    'description': 'Pyterlei, invista no seu futuro! Os investimentos são essenciais para o crescimento do seu patrimônio e realização de seus sonhos. Não deixe para depois!'}]},
 {'id': 2,
  'name': 'Pytano',
  'account': {'id': 7,
   'number': '00002-1',
   'agency': '0001',
   'balance': 500.0,
   'limit': 500.0},
  'card': {'id': 2, 'number': '**** **** **** 1112', 'limit': 1000.0},
  'features': [],
  'news': [{'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg',
    'description': 'Pytano, invista agora para proteger e fazer crescer seu patrimônio. Garanta seu futuro financeiro com as melhores opções de inve